# Open AI 모델 파인튜닝

이 노트북은 Open AI의 [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) 문서에서 제공하는 최신 가이드를 기반으로 작성되었습니다.

파인튜닝은 특정 사용 사례나 시나리오에 맞는 추가 데이터와 컨텍스트로 모델을 재학습시켜, 기본 모델의 성능을 애플리케이션에 맞게 향상시킵니다. _few shot learning_이나 _retrieval augmented generation_과 같은 프롬프트 엔지니어링 기법을 사용하면, 관련 데이터를 프롬프트에 추가해 품질을 높일 수 있습니다. 하지만 이런 방식은 대상 모델의 최대 토큰 윈도우 크기에 제한을 받습니다.

파인튜닝을 사용하면, 실제로 필요한 데이터를 활용해 모델 자체를 재학습시키게 됩니다(최대 토큰 윈도우에 들어갈 수 있는 것보다 훨씬 더 많은 예시를 사용할 수 있음). 그리고 추론 시점에 예시를 따로 제공하지 않아도 되는 _커스텀_ 모델 버전을 배포할 수 있습니다. 이렇게 하면 프롬프트 설계의 효율성이 높아질 뿐만 아니라(토큰 윈도우를 다른 용도로 더 자유롭게 활용 가능), 추론 시 모델에 보내야 하는 토큰 수가 줄어들어 비용도 절감될 수 있습니다.

파인튜닝은 4단계로 진행됩니다:
1. 학습 데이터를 준비하고 업로드합니다.
1. 학습 작업을 실행해 파인튜닝된 모델을 만듭니다.
1. 파인튜닝된 모델을 평가하고 품질을 높이기 위해 반복합니다.
1. 만족스러울 때 파인튜닝된 모델을 추론용으로 배포합니다.

모든 기본 모델이 파인튜닝을 지원하는 것은 아니니, 최신 정보는 [OpenAI 문서](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst)를 참고하세요. 이전에 파인튜닝한 모델을 다시 파인튜닝하는 것도 가능합니다. 이 튜토리얼에서는 `gpt-35-turbo`를 파인튜닝 대상 모델로 사용할 예정입니다.

---


### 1.1단계: 데이터셋 준비하기

주기율표의 원소에 대해 질문하면 라임이 들어간 짧은 시(리머릭)로 답해주는 챗봇을 만들어봅시다. _이번_ 간단한 튜토리얼에서는, 모델을 학습시키기 위해 데이터의 기대 형식을 보여주는 몇 가지 예시 응답으로 데이터셋을 만들어볼 거예요. 실제로 사용하려면 훨씬 더 많은 예시가 담긴 데이터셋이 필요합니다. 만약 해당 분야에 맞는 공개 데이터셋이 있다면, 그것을 재구성해서 파인튜닝에 활용할 수도 있습니다.

우리는 `gpt-35-turbo`에 집중하고, 한 번의 질문에 한 번 답하는(단일 턴) 챗 컴플리션을 원하므로, OpenAI 챗 컴플리션 요구사항을 반영한 [이 추천 포맷](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst)을 사용해 예시를 만들 수 있습니다. 만약 여러 번의 대화가 오가는 시나리오라면, [멀티턴 예시 포맷](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst)을 사용해야 하며, 이 포맷에는 파인튜닝 과정에서 어떤 메시지를 사용할지(또는 사용하지 않을지) 알려주는 `weight` 파라미터가 포함되어 있습니다.

여기서는 더 간단한 단일 턴 포맷을 사용할 거예요. 데이터는 [jsonl 포맷](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst)으로, 한 줄에 하나의 레코드가 들어가고, 각 레코드는 JSON 형식의 객체로 표현됩니다. 아래 예시는 2개의 레코드를 보여주고 있습니다. 전체 샘플 세트(10개 예시)는 [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl)에서 확인할 수 있습니다. **참고:** 각 레코드는 반드시 한 줄에 정의되어야 하며(일반적인 JSON 파일처럼 여러 줄로 나누면 안 됩니다)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

실제 사용에서는 좋은 결과를 얻으려면 훨씬 더 많은 예시가 필요합니다. 응답의 품질과 파인튜닝에 드는 시간/비용 사이에서 균형을 잡아야 하죠. 우리는 과정을 빠르게 보여주기 위해 작은 데이터셋을 사용합니다. 더 복잡한 파인튜닝 튜토리얼은 [이 OpenAI Cookbook 예시](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst)를 참고하세요.


### 1.2단계 데이터셋 업로드하기

Files API를 사용해 데이터를 업로드하세요 [여기에서 설명된 대로](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). 이 코드를 실행하려면 먼저 다음 단계를 완료해야 합니다:
 - `openai` Python 패키지를 설치했는지 확인하세요 (최신 기능을 사용하려면 버전이 0.28.0 이상이어야 합니다)
 - `OPENAI_API_KEY` 환경 변수를 본인의 OpenAI API 키로 설정하세요
자세한 내용은 강의에서 제공하는 [설정 가이드](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst)를 참고하세요.

이제, 로컬 JSONL 파일에서 업로드할 파일을 생성하는 코드를 실행하세요.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### 2.1단계: SDK로 파인튜닝 작업 생성하기


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### 2.2단계: 작업 상태 확인하기

`client.fine_tuning.jobs` API로 할 수 있는 몇 가지 작업은 다음과 같습니다:
- `client.fine_tuning.jobs.list(limit=<n>)` - 최근 n개의 파인튜닝 작업 목록 보기
- `client.fine_tuning.jobs.retrieve(<job_id>)` - 특정 파인튜닝 작업의 세부 정보 확인
- `client.fine_tuning.jobs.cancel(<job_id>)` - 파인튜닝 작업 취소하기
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - 해당 작업에서 최대 n개의 이벤트 목록 보기
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

이 과정의 첫 번째 단계는 _학습 파일을 검증_ 해서 데이터 형식이 올바른지 확인하는 것입니다.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### 2.3단계: 진행 상황을 모니터링하기 위해 이벤트 추적하기


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### 2.4단계: OpenAI 대시보드에서 상태 확인


OpenAI 웹사이트에서 _Fine-tuning_ 섹션을 확인하면 상태를 볼 수 있습니다. 여기서 현재 작업의 상태를 확인할 수 있고, 이전 작업 실행 내역도 추적할 수 있습니다. 이 스크린샷에서는 이전 실행이 실패했고, 두 번째 실행이 성공한 것을 볼 수 있습니다. 참고로, 첫 번째 실행에서는 JSON 파일의 레코드 형식이 잘못되어 실패했지만, 수정 후 두 번째 실행은 정상적으로 완료되어 모델을 사용할 수 있게 되었습니다.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.ko.png)


아래와 같이 시각적 대시보드에서 더 아래로 스크롤하면 상태 메시지와 지표도 확인할 수 있습니다:

| 메시지 | 지표 |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.ko.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.ko.png)|


### 3.1단계: ID 가져오기 및 코드에서 파인튜닝된 모델 테스트하기


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### 3.2단계: Playground에서 미세 조정된 모델 불러오기 및 테스트

이제 미세 조정된 모델을 두 가지 방법으로 테스트할 수 있습니다. 먼저, Playground에 접속해서 Models 드롭다운에서 새로 미세 조정한 모델을 선택할 수 있습니다. 또 다른 방법은 Fine-tuning 패널에 표시된 "Playground" 옵션을 사용하는 것으로(위 스크린샷 참고), 이를 클릭하면 아래와 같은 _비교_ 화면이 열려서 기본 모델과 미세 조정 모델의 결과를 나란히 볼 수 있어 빠르게 평가할 수 있습니다.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.ko.png)

학습 데이터에 사용한 시스템 컨텍스트를 입력하고 테스트할 질문을 작성하면 됩니다. 양쪽 모두 동일한 컨텍스트와 질문으로 업데이트되는 것을 볼 수 있습니다. 비교를 실행하면 두 모델의 출력 결과 차이를 확인할 수 있습니다. _미세 조정된 모델이 예시에서 제공한 형식대로 응답을 생성하는 반면, 기본 모델은 시스템 프롬프트만 따르는 점에 주목하세요._

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.ko.png)

비교 화면에서는 각 모델의 토큰 수와 추론에 걸린 시간도 함께 제공됩니다. **이 예시는 과정을 보여주기 위한 단순한 예시일 뿐 실제 데이터셋이나 상황을 반영하지는 않습니다.** 두 샘플 모두 토큰 수가 동일하게 표시되는 것을 볼 수 있는데(시스템 컨텍스트와 사용자 프롬프트가 동일하기 때문), 미세 조정 모델(커스텀 모델)이 추론에 더 많은 시간이 소요되는 것도 확인할 수 있습니다.

실제 환경에서는 이런 단순 예시가 아니라 실제 데이터(예: 고객 서비스용 상품 카탈로그 등)를 기반으로 미세 조정하게 되며, 이때 응답 품질의 차이가 훨씬 뚜렷하게 나타납니다. _이런 상황에서는 기본 모델로 동일한 품질의 응답을 얻으려면 프롬프트 엔지니어링을 더 많이 해야 하고, 그만큼 토큰 사용량과 추론 시간도 늘어날 수 있습니다._ _직접 시도해보고 싶다면 OpenAI Cookbook의 미세 조정 예시를 참고해 시작해 보세요._



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확한 내용이 포함될 수 있습니다. 원본 문서(원어)가 공식적인 기준임을 유의해 주시기 바랍니다. 중요한 정보의 경우, 전문 번역가에 의한 번역을 권장합니다. 본 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.
